# Lexy client

In [ ]:
from lexy_py import LexyClient

lexy = LexyClient()

In [ ]:
import inspect
import pytest

# Collections

### List collections

In [ ]:
lexy.list_collections()

### Get collection

In [ ]:
default_collection = lexy.get_collection('default')
default_collection

In [ ]:
default_docs = default_collection.list_documents()
default_docs[:5]

In [ ]:
doc = default_docs[0]
doc

### Create a new collection

In [ ]:
from lexy_py.collection.models import Collection

In [ ]:
print(inspect.signature(Collection))

In [ ]:
collection_without_a_client = Collection(collection_id='notbeingcreated')
collection_without_a_client

In [ ]:
with pytest.raises(ValueError) as exc_info:
    collection_without_a_client.list_documents()
print(exc_info)
assert isinstance(exc_info.value, ValueError)
assert str(exc_info.value) == "API client has not been set."

In [ ]:
new_collection = lexy.create_collection(collection_id='junk', description='just testing for now')
new_collection

In [ ]:
lexy.list_collections()

In [ ]:
new_collection.list_documents()

In [ ]:
lexy.delete_collection('junk')

In [ ]:
lexy.list_collections()

# Documents

### List documents

In [ ]:
lexy.list_documents()[:5]

In [ ]:
code_docs = lexy.list_documents('code')
code_docs

In [ ]:
code_doc = code_docs[0]
code_doc

In [ ]:
assert isinstance(code_doc.client, LexyClient)

### Get document

In [ ]:
sample_doc = lexy.get_document(code_doc.document_id)
sample_doc

In [ ]:
sample_doc.dict()

In [ ]:
assert isinstance(sample_doc.client, LexyClient)

### Create new document

In [ ]:
from lexy_py.document.models import Document

In [ ]:
print(inspect.signature(Document))

In [ ]:
Document("this is a new doc")

In [ ]:
new_docs = lexy.add_documents([
    {'content': 'This is my shiny new document!'}
])
new_doc = new_docs[0]
new_doc

In [ ]:
lexy.delete_document(document_id=new_doc.document_id)

# Indexes

### List indexes

In [ ]:
lexy.list_indexes()

### Get index

In [ ]:
idx = lexy.get_index('default_text_embeddings')
idx

In [ ]:
idx.query('hello world')

### Create new index

In [ ]:
from lexy_py.index.models import Index

In [ ]:
print(inspect.signature(Index))

In [ ]:
# TODO: implement this after setting up mock (do not run against live server)

# Bindings

### List bindings

In [ ]:
lexy.list_bindings()

### Get binding

In [ ]:
binding = lexy.get_binding(1)
binding

In [ ]:
binding.collection

In [ ]:
binding.collection.description

In [ ]:
binding.transformer

In [ ]:
binding.index

In [ ]:
binding.index.description

In [ ]:
binding.index.query('hi')

In [ ]:
binding.collection.list_documents()[:5]

In [ ]:
assert isinstance(binding.collection.client, LexyClient)

### Create new binding

In [ ]:
from lexy_py.binding.models import TransformerIndexBindingCreate

In [ ]:
print(inspect.signature(TransformerIndexBindingCreate))

In [ ]:
# TODO: implement this after setting up mock (do not run against live server)